# Minimax Engine with Alpha-Beta Pruning for Simplified Dominion
Daniel Brooks  
August 14, 2021  
danielbrooks20@gmail.com  

Here, we present a simplified version of the deckbuilding game Dominion. There are two players, each of whom start with 5 coins/turn. Each turn a player can spend 5 coins to generate +1 coin/turn, spend 5 coins for +3 points or spend 8 coins to buy a Province for +6 points. There are only 8 Provinces. When the last is purchsed, the player with the most points wins (if points are tied, the game is a draw). 

In tournament dominion, it's common for players to check if they have an immediate win, then prevent their opponent from having an immediate win, and so on. This idea is implemented here with the minimax algorithm (with alpha-beta pruning of unused nodes for efficiency). The competing minimax players will look a certain number of moves ahead (depth) for a forced win. If a forced win is not found, they will instead optimize for a heuristic evaluation of the position (more coins and points). 

The agents aren't perfect (because the depth is finite and the heuristic evaluation is imperfect), but they are pretty good. The winning agent will win the game as quickly as possible. The losing agent will attempt to drag the game on. Concepts like the penultimate Province rule are respected. Because this simplified implementation has infinite Duchies, some games will continue indefinitely!

This method shows great promise for improving existing dominion bots, enabling potential forced wins over multiple turns. Some modification will be required to handle the increased decision space and variance in card ordering.

You can make the minimax players battle it out in different scenarios at the bottom of the notebook. Enjoy!

Reference: https://stackabuse.com/minimax-and-alpha-beta-pruning-in-python

### Imports

In [1]:
import math
from typing import Dict, List, Tuple

def sigmoid(x: float) -> float:
    return 1 / (1 + math.exp(-x))

### Enumerating available purchases

In [2]:
dp = {} # Keep this forever.

def get_valid_buys(coins_left: int, provinces_left: int, dp: Dict) -> List:
    """
    Return a list of possible buys.
    
    Will return a list of lists of length 3 corresponding to [# Province, # Duchy, # Econ]
    
    For 7 coins will return:
    [[0, 1, 2], [0, 0, 7]]
    
    Possible optimization - more recursion to improve cache hit rate. 
    """
    # Trick to increase cache hit rate.
    if provinces_left > coins_left // 8:
        provinces_left = coins_left // 8
    
    if (coins_left, provinces_left) in dp:
        return dp[(coins_left, provinces_left)]
    
    possible_buys = []
    for province_buys in range(0, min(coins_left // 8, provinces_left) + 1):
        coins_after_provinces = coins_left - province_buys * 8
        for duchy_buys in range((coins_after_provinces // 5) + 1):
            coins_after_duchies = coins_after_provinces - 5 * duchy_buys
            possible_buys.append([province_buys, duchy_buys, coins_after_duchies])
            
    dp[(coins_left, provinces_left)] = possible_buys
    
    return possible_buys

### Minimax agents and game implementation

In [3]:
class Game:
    def __init__(self):
        self.MAX_DEPTH = 8
        self.MAX_TURNS = 20
        
    def new_game(self, state_dict: Dict = None) -> None:
        if state_dict:
            self.sd = state_dict
        else:
            self.sd = {"p1_point_lead" : 0, "p1_coins": 5, "p2_coins": 5, "provinces_left" : 8}
        self.current_turn = 1
        self.p1_turn = True
        self.minimax_winrate = 0.5
        print("Started new game.\n")
    
    def print_minimax_winrate(self) -> str:
        if abs(self.minimax_winrate) < 1000:
            return f"{self.minimax_winrate:.5f}"
        elif self.minimax_winrate >= 1000:
            return f"P1 wins in {abs(self.minimax_winrate - 1000 - self.MAX_DEPTH + 1)}"
        elif self.minimax_winrate <= -1000:
            return f"P2 wins in {abs(self.minimax_winrate + 1000 + self.MAX_DEPTH - 1)}"
        
    
    def p1_win_odds_heuristic(self) -> float:
        
        # Value per point in deck. 
        marginal_point_value = 0.2 * 9 / (self.sd["provinces_left"] + 5)
        
        # Value per unit of econ in deck. 
        marginal_econ_value = 1.0
        
        # On player 1's turn, incorporate P2's best move to smooth out winrate estimate.
        if self.p1_turn:
            vp_per_coin = 8.0 / 6
            econ_per_coin = 0.2
            initiative_value = max(vp_per_coin * marginal_point_value * self.sd["p2_coins"], 
                                   econ_per_coin * marginal_econ_value * self.sd["p2_coins"])
        else:
            initiative_value = 0
        
        point_value = self.sd["p1_point_lead"] * marginal_point_value
        coin_value =(self.sd["p1_coins"] - self.sd["p2_coins"]) * 1.0 * marginal_econ_value
        return sigmoid(0.5 * (point_value + coin_value + initiative_value))
    
    
    def print_state(self) -> None:
        print(f"Turn {int(self.current_turn)}, P{2-int(self.p1_turn)}'s turn")
        print(f"Heuristic wr: {self.p1_win_odds_heuristic():.5f}, Minimax winrate: {self.print_minimax_winrate()}")
        print(f"Game state: {self.sd}")
        
    def is_end(self) -> str:
        if self.sd["provinces_left"] == 0:
            if self.sd["p1_point_lead"] > 0:
                return "Player 1 wins!"
            elif self.sd["p1_point_lead"] < 0:
                return "Player 2 wins!"
            else:
                return "Tie game!"
        
    def make_buy(self, buy: List, player1: bool, reverse=False) -> None:
        """
        Make a buy and update the current gamestate.
        
        If reverse=True, that buy will be reverted.
        """
        if reverse:
            rev = -1
        else:
            rev = 1
        
        self.sd["provinces_left"] -= buy[0] * rev
        
        vp_gained = 6 * buy[0] + 3 * buy[1]
        econ_gained = buy[2] // 5
        
        if player1:
            self.sd["p1_point_lead"] += vp_gained * rev
            self.sd["p1_coins"] += econ_gained * rev
        else:
            self.sd["p1_point_lead"] -= vp_gained * rev
            self.sd["p2_coins"] += econ_gained * rev
            
    def get_result(self, depth=0) -> Tuple:
        # Quicker wins are better, when possible.
        result = self.is_end()
        
        if result == "Player 1 wins!":
            return (1000 + depth, "Does not matter - p1 wins") # does this work?
        elif result == "Player 2 wins!":
            return (-1000 - depth, "Does not matter - p2 wins")
        elif result == "Tie game!":
            return (0.5, "Does note matter, tie game")
        
    def print_buy(self, buy: str, player1: bool=True) -> None:
        print(f"Player{1 if player1 else 2} bought {buy[0]} Provinces, {buy[1]} Duchies, and +{buy[2]//5} coins.")
    
    def play_alpha_beta(self, state_dict:Dict=None) -> None:
        """
        Start a new game between max and min players. 
        """
        self.new_game(state_dict)
        
        # Keep taking turns until the game is over. 
        while True:
                        
            # Check for a winner. 
            self.print_state()
            self.result = self.is_end()
            if self.result != None:
                print(self.result)
                return
            
            # Have P1 make a move. 
            if self.p1_turn:
                (max_wr, max_buy) = self.max_alpha_beta(-math.inf, math.inf, self.MAX_DEPTH)
                self.minimax_winrate = max_wr
                self.make_buy(buy=max_buy, player1=True)
                self.p1_turn = False
                self.print_buy(max_buy)
                
                
            # Have P2 make a move. 
            else:
                (min_wr, min_buy) = self.min_alpha_beta(-math.inf, math.inf, self.MAX_DEPTH)
                self.minimax_winrate = min_wr
                self.make_buy(buy=min_buy, player1=False)
                self.p1_turn = True
                self.print_buy(min_buy, player1=False)
                
            # Between turns.
            self.current_turn += 0.5
            print("\n")
            if self.current_turn > self.MAX_TURNS:
                print("Max turns reached. Stopping.")
                break
            
    def max_alpha_beta(self, alpha: float, beta: float, depth: int) -> Tuple:
        """
        Have player 1 pick a move. 
        """
        value = -math.inf
                
        # If game is over, return that state. 
        result = self.get_result(depth)
        if result:
            return result
        
        # If we're at max depth, return heuristic value. 
        if depth == 0:
            return (self.p1_win_odds_heuristic(), "Does not matter, max depth, P1.")
        
        # Come up with a list of possible buys. 
        candidate_buys = get_valid_buys(self.sd["p1_coins"], self.sd["provinces_left"], dp)
        for buy in candidate_buys:
            
            # Update state. 
            self.make_buy(buy=buy, player1=True)

            # Alpha beta search here. 
            (min_wr, min_buy) = self.min_alpha_beta(alpha, beta, depth - 1)
            if min_wr > value:
                best_buy = buy
                value = min_wr

            # Revert state. 
            self.make_buy(buy=buy, player1=True, reverse=True)
            
            # Prune suboptimal buys. 
            if value >= beta:
                return (value, best_buy)
            alpha = max(alpha, value)
        
        return (value, best_buy)
    
    
    def min_alpha_beta(self, alpha: float, beta: float, depth: int) -> Tuple:
        """
        Have player 2 pick a move. 
        """
        value = math.inf
                
        # If game is over, return that state. 
        result = self.get_result(depth)
        if result:
            return result

        # If we're at max depth, return heuristic value. 
        if depth == 0:
            return (self.p1_win_odds_heuristic(), "Does not matter, max depth, P2.")
        
        # Come up with a list of possible buys. 
        candidate_buys = get_valid_buys(self.sd["p2_coins"], self.sd["provinces_left"], dp)  
        for buy in candidate_buys:
            
            # Update state. 
            self.make_buy(buy=buy, player1=False)

            # Alpha beta search here. 
            (max_wr, max_buy) = self.max_alpha_beta(alpha, beta, depth - 1)
            
            if max_wr < value:
                best_buy = buy
                value = max_wr

            # Revert state. 
            self.make_buy(buy=buy, player1=False, reverse=True)
            
            # Prune suboptimal buys. 
            if value <= alpha:
                return (value, best_buy)
            beta = min(beta, value)
            
        return (value, best_buy)

### Watch the agents play

In [4]:
# Modify the parameters in self.sd to get different games. 
g = Game()
g.play_alpha_beta(
    state_dict = {"p1_point_lead" : 0, "p1_coins": 12, "p2_coins": 8, "provinces_left" : 8}
)

Started new game.

Turn 1, P1's turn
Heuristic wr: 0.94268, Minimax winrate: 0.50000
Game state: {'p1_point_lead': 0, 'p1_coins': 12, 'p2_coins': 8, 'provinces_left': 8}
Player1 bought 0 Provinces, 0 Duchies, and +2 coins.


Turn 1, P2's turn
Heuristic wr: 0.95257, Minimax winrate: 0.99597
Game state: {'p1_point_lead': 0, 'p1_coins': 14, 'p2_coins': 8, 'provinces_left': 8}
Player2 bought 0 Provinces, 0 Duchies, and +1 coins.


Turn 2, P1's turn
Heuristic wr: 0.96770, Minimax winrate: 0.99753
Game state: {'p1_point_lead': 0, 'p1_coins': 14, 'p2_coins': 9, 'provinces_left': 8}
Player1 bought 0 Provinces, 0 Duchies, and +2 coins.


Turn 2, P2's turn
Heuristic wr: 0.97069, Minimax winrate: 0.99911
Game state: {'p1_point_lead': 0, 'p1_coins': 16, 'p2_coins': 9, 'provinces_left': 8}
Player2 bought 0 Provinces, 0 Duchies, and +1 coins.


Turn 3, P1's turn
Heuristic wr: 0.98201, Minimax winrate: P1 wins in 7
Game state: {'p1_point_lead': 0, 'p1_coins': 16, 'p2_coins': 10, 'provinces_left': 8}
